# AI-PSCI-009: Protein Data Acquisition & Target Selection

**AI in Pharmaceutical Sciences: Bench to Bedside**  
VCU School of Pharmacy | VIP Program | Spring 2026

---

**Week 5 | Module: AI Tools Overview | Estimated Time: 60-90 minutes**

**Prerequisites**: AI-PSCI-001 through AI-PSCI-008

---

**🎯 IMPORTANT: This talktorial marks the beginning of YOUR research project!**

You will select a drug target to study for the rest of the semester. All subsequent talktorials will use YOUR chosen target. Choose wisely — this will be the foundation of your capstone project!

## 🎯 Learning Objectives

After completing this talktorial, you will be able to:

1. Select a drug target from the 6-target portfolio and justify your choice
2. Query the UniProt database to retrieve protein sequences (FASTA format)
3. Download and parse protein structures from the Protein Data Bank (PDB)
4. Map between different database identifiers (UniProt, PDB, ChEMBL)
5. Understand the relationship between protein sequence and structure

---

## 📚 Background

### From Small Molecules to Proteins

In Weeks 1-4, you learned to work with **small molecules** — drugs that bind to protein targets. Now we shift focus to the **proteins themselves**. Understanding protein structure is essential for:

- **Structure-based drug design**: Designing molecules that fit protein binding sites
- **Resistance prediction**: Understanding how mutations affect drug binding
- **Target validation**: Confirming a protein is a viable drug target

### Key Biological Databases

**UniProt** (Universal Protein Resource)
- Comprehensive protein sequence database
- Each protein has a unique accession number (e.g., P0ABQ4 for E. coli DHFR)
- Contains sequence, function, variants, and cross-references
- URL: https://www.uniprot.org

**PDB** (Protein Data Bank)
- Repository of 3D protein structures determined by X-ray, NMR, or cryo-EM
- Each structure has a 4-character ID (e.g., 1RX1 for DHFR with trimethoprim)
- Structures include atomic coordinates, ligands, and metadata
- URL: https://www.rcsb.org

**AlphaFold Database**
- AI-predicted structures for nearly all known proteins
- Based on AlphaFold2's revolutionary structure prediction
- Includes confidence scores (pLDDT) for each residue
- URL: https://alphafold.ebi.ac.uk

### Key Concepts

- **FASTA format**: Text format for protein sequences (header line + sequence)
- **PDB format**: Text format for 3D atomic coordinates
- **Cross-references**: Links between databases (UniProt ↔ PDB ↔ ChEMBL)
- **Sequence vs Structure**: Sequence = 1D amino acid chain; Structure = 3D fold

---

## 🛠️ Setup

Run this cell to install required packages:

In [ ]:
#@title 🛠️ Install Packages
!pip install biopython py3Dmol requests -q
print("✅ Packages installed successfully!")

Import the required libraries:

In [ ]:
#@title 📦 Import Libraries
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.PDB import PDBParser, PDBIO, Select
from io import StringIO
import py3Dmol
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', 10)
plt.rcParams['figure.figsize'] = [10, 6]

print("✅ All libraries imported!")

---

## 🎯 Target Selection

**It's time to choose your research target!** You will work with this target for the rest of the semester (Weeks 5-16).

### Available Targets

| Target | Disease Area | Why It's Interesting | Complexity |
|--------|-------------|---------------------|------------|
| **DHFR** (default) | Antibiotic resistance | Classic drug target, well-studied, resistance mutations | ⭐ Beginner-friendly |
| **ABL1** | Cancer (CML) | Gleevec revolution — changed cancer treatment forever | ⭐⭐ Intermediate |
| **EGFR** | Cancer (NSCLC) | Targeted cancer therapy, precision medicine poster child | ⭐⭐ Intermediate |
| **AChE** | Alzheimer's | Aging population crisis, deep binding pocket | ⭐⭐ Intermediate |
| **COX-2** | Pain/Inflammation | Most common drug class (NSAIDs), selectivity story | ⭐⭐ Intermediate |
| **DPP-4** | Type 2 Diabetes | 1 in 10 adults have diabetes, "gliptin" drugs | ⭐⭐ Intermediate |

### Considerations for Your Choice

- **Personal interest**: What disease area motivates you?
- **Career goals**: What therapeutic area might you work in?
- **Data availability**: All targets have good data, but some have more compounds (AChE, DPP-4)
- **Complexity**: DHFR is smallest (159 aa), recommended for beginners

### Make Your Selection Below

In [ ]:
#@title 🎯 Select Your Drug Target

TARGET = "DHFR" #@param ["DHFR", "ABL1", "EGFR", "AChE", "COX-2", "DPP-4"]

# Complete target configuration
TARGET_CONFIG = {
    "DHFR": {
        "pdb": "1RX1",
        "uniprot": "P0ABQ4",
        "chembl": "CHEMBL202",
        "drug": "Trimethoprim",
        "drug_chembl": "CHEMBL22",
        "organism": "Escherichia coli",
        "full_name": "Dihydrofolate reductase",
        "disease": "Bacterial infections (antibiotic resistance)"
    },
    "ABL1": {
        "pdb": "1IEP",
        "uniprot": "P00519",
        "chembl": "CHEMBL1862",
        "drug": "Imatinib",
        "drug_chembl": "CHEMBL941",
        "organism": "Homo sapiens",
        "full_name": "Tyrosine-protein kinase ABL1",
        "disease": "Chronic myeloid leukemia (CML)"
    },
    "EGFR": {
        "pdb": "1M17",
        "uniprot": "P00533",
        "chembl": "CHEMBL203",
        "drug": "Erlotinib",
        "drug_chembl": "CHEMBL553",
        "organism": "Homo sapiens",
        "full_name": "Epidermal growth factor receptor",
        "disease": "Non-small cell lung cancer (NSCLC)"
    },
    "AChE": {
        "pdb": "4EY7",
        "uniprot": "P22303",
        "chembl": "CHEMBL220",
        "drug": "Donepezil",
        "drug_chembl": "CHEMBL502",
        "organism": "Homo sapiens",
        "full_name": "Acetylcholinesterase",
        "disease": "Alzheimer's disease"
    },
    "COX-2": {
        "pdb": "3LN1",
        "uniprot": "P35354",
        "chembl": "CHEMBL230",
        "drug": "Celecoxib",
        "drug_chembl": "CHEMBL118",
        "organism": "Homo sapiens",
        "full_name": "Prostaglandin G/H synthase 2",
        "disease": "Inflammation and pain"
    },
    "DPP-4": {
        "pdb": "1X70",
        "uniprot": "P27487",
        "chembl": "CHEMBL284",
        "drug": "Sitagliptin",
        "drug_chembl": "CHEMBL1422",
        "organism": "Homo sapiens",
        "full_name": "Dipeptidyl peptidase 4",
        "disease": "Type 2 diabetes"
    }
}

# Get configuration for selected target
config = TARGET_CONFIG[TARGET]

print("=" * 60)
print(f"🎯 Your Selected Target: {TARGET}")
print("=" * 60)
print(f"\n🧬 Full Name: {config['full_name']}")
print(f"🦠 Organism: {config['organism']}")
print(f"🏥 Disease Context: {config['disease']}")
print(f"\n📊 Database Identifiers:")
print(f"   PDB: {config['pdb']}")
print(f"   UniProt: {config['uniprot']}")
print(f"   ChEMBL Target: {config['chembl']}")
print(f"\n💊 Reference Drug: {config['drug']} (ChEMBL: {config['drug_chembl']})")
print("\n✅ Target configuration saved!")

---

## 🔬 Guided Inquiry 1: Fetching Protein Sequence from UniProt

### Context

The **protein sequence** is the foundation of all structural biology. It defines the amino acids that fold into the 3D structure. UniProt provides sequences in **FASTA format** — a simple text format used throughout bioinformatics.

FASTA format:
```
>sp|P0ABQ4|DHFR_ECOLI Dihydrofolate reductase OS=Escherichia coli
MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVIMGRHTWESIGRPLPGRKNI
ILSSQPGTDDRVTWVKSVDEAIAACGDVPEIMVIGGGRVYEQFLPKAQKLYLTHIDAEVEG
DTHFPDYEPDDWESVFSEFHDADAQNSHSYCFEILERR
```

### Your Task

Using your AI assistant, write code to:

1. Construct the UniProt REST API URL for your target's sequence
2. Fetch the FASTA sequence using the `requests` library
3. Parse the sequence using BioPython's `SeqIO`
4. Display the sequence length and first/last 10 amino acids

💡 **Prompting Tips**:
- Ask: "How do I fetch a protein sequence from UniProt using Python requests?"
- The UniProt REST API URL pattern is: `https://rest.uniprot.org/uniprotkb/{UNIPROT_ID}.fasta`
- Your UniProt ID is stored in `config['uniprot']`

### Verification

After running your code, confirm:

| Target | Expected Sequence Length |
|--------|-------------------------|
| DHFR | 159 aa |
| ABL1 | 1130 aa |
| EGFR | 1210 aa |
| AChE | 614 aa |
| COX-2 | 604 aa |
| DPP-4 | 766 aa |

- [ ] Sequence length matches expected value
- [ ] FASTA header contains protein name and organism
- [ ] Sequence contains only standard amino acid letters

📓 **Lab Notebook**: Record your target's sequence length and the first 10 amino acids. Compare to other students — which targets are largest/smallest?

In [ ]:
# Your code here



---

## 🔬 Guided Inquiry 2: Downloading Structure from PDB

### Context

While sequences are 1D, **protein structures** provide the 3D atomic coordinates essential for drug design. The Protein Data Bank (PDB) hosts experimentally determined structures from X-ray crystallography, NMR, and cryo-EM.

Our reference PDB structures include the protein **bound to a drug** — these co-crystal structures show exactly how drugs interact with the binding site.

### Your Task

Using your AI assistant, write code to:

1. Construct the PDB download URL for your target's structure
2. Download the PDB file using the `requests` library
3. Parse the structure using BioPython's PDBParser
4. Report: number of chains, residues, and atoms

💡 **Prompting Tips**:
- Ask: "How do I download a PDB file using Python?"
- PDB download URL: `https://files.rcsb.org/download/{PDB_ID}.pdb`
- Your PDB ID is stored in `config['pdb']`

### Verification

After running your code, confirm:
- [ ] PDB file downloads successfully
- [ ] Structure contains at least one protein chain
- [ ] Number of residues is close to sequence length
- [ ] Structure may include ligand (HETATM records)

📓 **Lab Notebook**: Record the number of chains in your structure. Is your protein a monomer or oligomer?

In [ ]:
# Your code here



---

## 🔬 Guided Inquiry 3: Visualizing Your Target Structure

### Context

Visualization helps us understand protein architecture and binding sites. **py3Dmol** provides interactive 3D visualization in Jupyter notebooks. We'll visualize your target with:
- Protein backbone as cartoon
- Ligand as sticks (if present)
- Color by secondary structure

### Your Task

Using your AI assistant, write code to:

1. Load the PDB structure into py3Dmol
2. Display protein as cartoon with rainbow coloring
3. Show any ligands as sticks in a different color
4. Rotate the view to see the binding site

💡 **Prompting Tips**:
- Ask: "How do I visualize a protein structure with py3Dmol?"
- Cartoon representation shows secondary structure (α-helices, β-sheets)
- Use `{hetflag: true}` to select ligands

### Verification

After running your code, confirm:
- [ ] 3D structure is displayed and can be rotated
- [ ] Protein secondary structure is visible
- [ ] Ligand (if present) is highlighted

📓 **Lab Notebook**: Take a screenshot of your target structure. What secondary structure elements do you see? Where is the binding site located?

In [ ]:
# Your code here



---

## 🔬 Guided Inquiry 4: Comparing Sequence to Structure

### Context

PDB structures sometimes differ from the reference UniProt sequence due to:
- **Missing residues**: Disordered regions not visible in X-ray
- **Mutations**: Engineered variants for crystallization
- **Tags**: Purification tags (His-tag, etc.)
- **Truncations**: Only part of the protein crystallized

Understanding these differences is crucial for comparing structures and designing experiments.

### Your Task

Using your AI assistant, write code to:

1. Extract the sequence from the PDB structure (from CA atoms)
2. Compare PDB sequence length to UniProt sequence length
3. Identify any missing residues at N-terminus or C-terminus
4. Calculate the coverage percentage

💡 **Prompting Tips**:
- Ask: "How do I extract the amino acid sequence from a PDB structure?"
- Three-letter to one-letter amino acid conversion is needed
- PDB residue numbering may not start at 1

### Verification

After running your code, confirm:
- [ ] PDB sequence extracted successfully
- [ ] Coverage percentage calculated
- [ ] Any missing residues identified

📓 **Lab Notebook**: What percentage of your target is resolved in the crystal structure? Are any important regions missing?

In [ ]:
# Your code here



---

## 🔬 Guided Inquiry 5: Fetching AlphaFold Predicted Structure

### Context

**AlphaFold2** revolutionized structural biology by accurately predicting protein structures from sequence alone. The **AlphaFold Database** (AFDB) provides pre-computed predictions for nearly all known proteins.

AlphaFold structures are useful when:
- No experimental structure exists
- You need the full-length protein (PDB may be truncated)
- You want to predict mutant structures

### Your Task

Using your AI assistant, write code to:

1. Construct the AlphaFold Database URL for your target
2. Download the AlphaFold predicted structure
3. Compare the number of residues to PDB and UniProt
4. Visualize both structures side by side (if time permits)

💡 **Prompting Tips**:
- Ask: "How do I download an AlphaFold structure for a UniProt ID?"
- AFDB URL pattern: `https://alphafold.ebi.ac.uk/files/AF-{UNIPROT_ID}-F1-model_v6.pdb`
- Note: Version numbers change over time (currently v6 as of 2025)

### Verification

After running your code, confirm:
- [ ] AlphaFold structure downloads successfully
- [ ] AlphaFold covers the full UniProt sequence
- [ ] Both structures can be visualized

📓 **Lab Notebook**: Does the AlphaFold structure cover regions missing from the PDB? Note the differences.

In [ ]:
# Your code here



---

## 🔬 Guided Inquiry 6: Creating a Target Summary Report

### Context

You now have comprehensive information about your target. Let's compile everything into a summary that will serve as a reference for future talktorials.

### Your Task

Using your AI assistant, write code to:

1. Create a summary DataFrame with all target information
2. Calculate amino acid composition of your target sequence
3. Identify the most common amino acids
4. Save the summary to a file for future reference

💡 **Prompting Tips**:
- Ask: "How do I calculate amino acid composition of a protein sequence?"
- Use Counter from collections for counting
- Consider which amino acids are important for the binding site

### Verification

After running your code, confirm:
- [ ] Summary contains all database identifiers
- [ ] Amino acid composition calculated
- [ ] Most common amino acids identified
- [ ] Summary file saved

📓 **Lab Notebook**: Save this summary in your lab notebook. You'll refer to it throughout the semester.

In [ ]:
# Your code here



---

## ✅ Checkpoint

Before moving on to the next talktorial, confirm you can:

- [ ] Access your target configuration using the `config` dictionary
- [ ] Fetch protein sequences from UniProt using the REST API
- [ ] Download and parse PDB structures using BioPython
- [ ] Visualize protein structures using py3Dmol
- [ ] Compare sequence vs structure (identify missing residues)
- [ ] Access AlphaFold predicted structures from AFDB

### Your lab notebook should include:

- [ ] Your target selection and reasoning
- [ ] Target's database identifiers (UniProt, PDB, ChEMBL)
- [ ] Sequence length and key amino acid composition
- [ ] Screenshot of the 3D structure
- [ ] Notes on any missing residues in the PDB structure

---

## 🤔 Reflection Questions

Answer these in your lab notebook:

1. **Target Selection**: Why did you choose your target? How does the disease area connect to your career interests?

2. **Sequence vs Structure**: Your PDB structure may have missing residues compared to UniProt. Why might this be? How could this affect drug design?

3. **AlphaFold vs Experimental**: When would you use an AlphaFold structure instead of an experimental PDB structure? What are the trade-offs?

---

## 📖 Further Reading

- [UniProt: The Universal Protein Resource](https://www.uniprot.org/help/about) - Understanding protein annotations
- [RCSB PDB: Understanding PDB Format](https://www.rcsb.org/docs/general-help/organization-of-3d-structures-in-the-protein-data-bank) - PDB file format details
- [AlphaFold Protein Structure Database](https://alphafold.ebi.ac.uk/) - AI-predicted structures
- [BioPython Tutorial](https://biopython.org/DIST/docs/tutorial/Tutorial.html) - Working with biological data in Python

---

## 🔗 Connection to Research

You've just completed a workflow used by drug discovery scientists worldwide:

1. **Target validation**: Confirming a protein is a viable drug target
2. **Structural analysis**: Understanding the 3D architecture for drug design
3. **Data integration**: Connecting sequence, structure, and activity databases

### What's Next?

In **AI-PSCI-010**, you will:
1. Explore advanced protein visualization techniques
2. Identify binding site residues in your target
3. Compare your target to related proteins
4. Prepare for structure prediction with AlphaFold2

All future talktorials will use YOUR selected target. You're now building a personalized drug discovery pipeline!

---

*AI-PSCI-009 Complete. Proceed to AI-PSCI-010: Protein Structure Visualization.*